In [1]:
# run this notebook from root directory

# based on: https://github.com/jpwhite3/northwind-SQLite3

In [3]:
from datetime import timedelta, datetime
from random import randint
from random import choice as rc
import sqlite3

In [4]:
# This function will return a random datetime between two datetime objects.
def random_date(start, end):
    return start + timedelta(seconds=randint(0, int((end - start).total_seconds())))


In [7]:
def initialize_database(db_name, sql_file):
    # Connect to SQLite database (creates the file if it doesn't exist)
    with sqlite3.connect(db_name) as conn:
        cursor = conn.cursor()
        
        # Read the SQL file
        with open(sql_file, 'r') as f:
            sql_script = f.read()
        
        # Execute the SQL script to create tables and populate data
        cursor.executescript(sql_script)
        
    print(f"Database '{db_name}' initialized successfully.")

In [1]:
# specified relative to the root directory
DB_PATH = r"data/northwind_example_db/sqlite_northwind.db"
SQL_FILE_PATH = r"data/northwind_example_db/create.sql"

In [10]:
# get files from repo: https://github.com/jpwhite3/northwind-SQLite3/tree/main
# specifically:
# SQL create file -> https://github.com/jpwhite3/northwind-SQLite3/blob/main/src/create.sql
# DB file should get created automatically

In [9]:
initialize_database(DB_PATH, SQL_FILE_PATH)

Database 'data/northwind_example_db/sqlite_northwind.db' initialized successfully.


In [11]:
# Connect to the DB
conn = sqlite3.connect(DB_PATH)
c = conn.cursor()


In [12]:
# ShipName, ShipAddress, ShipCity, ShipRegion, ShipPostalCode
c.execute(
    "select distinct ShipName, ShipAddress, ShipCity, ShipRegion, ShipPostalCode, ShipCountry from [Orders]"
)
locations = [(row[0], row[1], row[2], row[3], row[4], row[5]) for row in c.fetchall()]



In [13]:
# Customer.Id
c.execute("select distinct EmployeeId from [Employees]")
employees = [row[0] for row in c.fetchall()]

In [14]:
# Shipper.Id
c.execute("select distinct ShipperId from [Shippers]")
shippers = [row[0] for row in c.fetchall()]

In [15]:
# Customer.Id
c.execute("select distinct CustomerId from [Customers]")
customers = [row[0] for row in c.fetchall()]


In [16]:
# Create a bunch of new orders
for i in range(randint(15000, 16000)):
    sql = "INSERT INTO [Orders] (CustomerId, EmployeeId, OrderDate, RequiredDate, ShippedDate, ShipVia, Freight, ShipName, ShipAddress, ShipCity, ShipRegion, ShipPostalCode, ShipCountry) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    location = rc(locations)
    order_date = random_date(
        datetime.strptime("2012-07-10", "%Y-%m-%d"), datetime.today()
    )
    required_date = random_date(
        order_date, order_date + timedelta(days=randint(14, 60))
    )
    shipped_date = random_date(order_date, order_date + timedelta(days=randint(1, 30)))
    params = (
        rc(customers),  # CustomerId
        rc(employees),  # EmployeeId
        order_date,  # OrderDate
        required_date,  # RequiredDate
        shipped_date,  # ShippedDate
        rc(shippers),  # ShipVia
        0.00,  # Freight
        location[0],  # ShipName
        location[1],  # ShipAddress
        location[2],  # ShipCity
        location[3],  # ShipRegion
        location[4],  # ShipPostalCode
        location[5],  # ShipCountry
    )
    c.execute(sql, params)



C:\Users\Maciek\AppData\Local\Temp\ipykernel_18936\1996910790.py:27: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  c.execute(sql, params)


In [17]:
# Product.Id
c.execute("select distinct ProductId, UnitPrice from [Products]")
products = [(row[0], row[1]) for row in c.fetchall()]

In [18]:
# Order.Id
c.execute("select distinct OrderId from [Orders] where Freight = 0.00")
orders = [row[0] for row in c.fetchall()]

In [19]:
# Fill the order with items
for order in orders:
    used = []
    for x in range(randint(1, len(products))):
        sql = "INSERT INTO [Order Details] (OrderId, ProductId, UnitPrice, Quantity, Discount) VALUES (?, ?, ?, ?, ?)"
        control = 1
        while control:
            product = rc(products)
            if product not in used:
                used.append(product)
                control = 0
        params = (
            # "%s/%s" % (order, product[0]),
            order,  # OrderId
            product[0],  # ProductId
            product[1],  # UnitPrice
            randint(1, 50),  # Quantity
            0,  # Discount
        )
        c.execute(sql, params)


In [20]:
# c.execute('update [Order] set OrderDate = date(OrderDate), RequiredDate = date(RequiredDate), ShippedDate = date(ShippedDate)')
c.execute("select sum(Quantity)*0.25+10, OrderId from [Order Details] group by OrderId")
orders = [(row[0], row[1]) for row in c.fetchall()]
for order in orders:
    c.execute("update [Orders] set Freight=? where OrderId=?", (order[0], order[1]))


In [21]:
conn.commit()
conn.close()